# Machine Learning Homework 

In [ ]:
# Dependencies
import numpy as np
import pandas as pd
import warnings 
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.keras.__version__
warnings.simplefilter('ignore', FutureWarning)

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

# Select your features (columns)

In [ ]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 
       'koi_time0bk', 'koi_impact', 'koi_duration','koi_depth', 'koi_prad', 
       'koi_teq', 'koi_insol',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 
       'koi_srad', 'ra', 'dec',
       'koi_kepmag']]
selected_features

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
y=df['koi_disposition']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=42)

In [ ]:
X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Step #1  Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

#  Setup Neural Network models



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### First create a basic model with 19 inputs, 60 hidden notes (~ 19*3), and 3 outputs (Candidate, Confirmed, False Positive)


In [ ]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=60, activation='relu', input_dim=19))
model.add(Dense(units=3, activation='softmax'))

In [ ]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

### Second, create a deep learning neural network by adding an additional layer of 60 notes

In [ ]:
deep_model = Sequential()
deep_model.add(Dense(units=60, activation='relu', input_dim=19))
deep_model.add(Dense(units=60, activation='relu'))
deep_model.add(Dense(units=60, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))

In [ ]:
# Compile and fit the model
deep_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
deep_model.summary()

In [ ]:
deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

## Evaluate basic verus deep model

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

## Make Predictions


In [ ]:
encoded_predictions = deep_model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

# Save the Deep Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'neural_network.sav'
# joblib.dump(model, filename)
deep_model.save('neural_network.h5')